In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import re

import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
%matplotlib inline

Основная задача работы - проведение EDA датасета (рейтинг фильмов с базы filmtv) на предмет зависимости среднего рейтинга кинематографа от страны производства, жанра и времени выпуска.

In [3]:
# Загрузка данных
df = pd.read_csv('filmtv_movies.csv', sep = ',')

В целом, датасет хорошо подготовлен, за исключением нескольких опечаток. % NaN-значений в целевых категориях не высок, эти данные по просту не будут использованы в анализе.

In [4]:
df.head()

,filmtv_id,title,year,genre,duration,country,director,actors,avg_vote,votes,description,notes
0,2,Bugs Bunny's Third Movie: 1001 Rabbit Tales,1982,Animation,76,United States,"David Detiege, Art Davis, Bill Perez",NaN,7.7,30,"With two protruding incisors, a little crafty ...","These are many small independent stories, whic..."
1,3,18 anni tra una settimana,1991,Drama,98,Italy,Luigi Perelli,"Kim Rossi Stuart, Simona Cavallari, Ennio Fant...",7.0,3,"Samantha, not yet eighteen, abandons the comfo...","Luigi Perelli, the director of ""Piovra"", occas..."
2,17,Ride a Wild Pony,1976,Romantic,91,United States,Don Chaffey,"Michael Craig, John Meillon, Eva Griffith, Gra...",5.7,11,In the Australia of the pioneers a boy and a g...,"""Ecological"" fable with a happy ending not wit..."
3,18,Diner,1982,Comedy,95,United States,Barry Levinson,"Mickey Rourke, Steve Guttenberg, Ellen Barkin",7.2,15,Five boys from Baltimore are in the habit of m...,A cast of will be famous for Levinson's direct...
4,20,A che servono questi quattrini?,1942,Comedy,85,Italy,Esodo Pratelli,"Eduardo De Filippo, Peppino De Filippo, Clelia...",5.9,12,"With a trick, the Marquis Parascandolo pennile...",Taken from the theatrical piece by Armando Cur...


В датасете присутствуют пропущенные данные, а также поля, содержащие по нескольку значений. Nan-значения будут опущены, информация из составных ячеек далее извлекается парсингом текста.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52507 entries, 0 to 52506
Data columns (total 12 columns):
filmtv_id      52507 non-null int64
title          52507 non-null object
year           52507 non-null int64
genre          52308 non-null object
duration       52507 non-null int64
country        52464 non-null object
director       52451 non-null object
actors         49685 non-null object
avg_vote       52507 non-null float64
votes          52507 non-null int64
description    148 non-null object
notes          133 non-null object
dtypes: float64(1), int64(4), object(7)
memory usage: 4.8+ MB


In [6]:
df.describe()

,filmtv_id,year,duration,avg_vote,votes
count,52507.000000,52507.000000,52507.000000,52507.000000,52507.000000
mean,42642.801588,1990.959434,98.522864,5.849411,24.266384
std,40193.994697,81.598024,35.247517,1.523136,60.186434
min,2.000000,1897.000000,40.000000,0.500000,1.000000
25%,14686.500000,1975.000000,89.000000,5.000000,2.000000
50%,30518.000000,1997.000000,95.000000,6.000000,4.000000
75%,55035.000000,2010.000000,105.000000,7.000000,18.000000
max,174891.000000,19942.000000,5280.000000,10.000000,1199.000000


In [7]:
df.isnull().sum()

filmtv_id          0
title              0
year               0
genre            199
duration           0
country           43
director          56
actors          2822
avg_vote           0
votes              0
description    52359
notes          52374
dtype: int64

In [25]:
# предварительная подгтовка данных
def prepare_data(df):
    df.loc[df['year']==19942, 'year']=1942
    df = df.dropna(subset=['genre'])
    return df

df = prepare_data(df)

Числовые показатели имеют слабую корреляцию

In [9]:
df.corr()

,filmtv_id,year,duration,avg_vote,votes
filmtv_id,1.000000,0.516086,-0.018504,-0.100780,-0.100569
year,0.516086,1.000000,0.042032,-0.156160,0.060738
duration,-0.018504,0.042032,1.000000,0.081358,0.129961
avg_vote,-0.100780,-0.156160,0.081358,1.000000,0.194235
votes,-0.100569,0.060738,0.129961,0.194235,1.000000


Средний рейтинг всего датасета составил 5.85 пунктов, дисперсия - 2.31, среднеквадратичное отклонение - 1.52

In [10]:
X = df.avg_vote

print(f'Среднее арифметическое: {np.mean(X)}')
print(f'Среднее квадратичное отклонение: {np.std(X)}')
print(f'Дисперсия: {np.var(X)}')

Среднее арифметическое: 5.848982947158782
Среднее квадратичное отклонение: 1.5216373945256327
Дисперсия: 2.315380360418756


За последние 60 лет выросло более, чем в 8 раз, и к 2016 г. превысило отметку в 1600. (в 2017-2019 гг. данные показали спад производства, что может быть следствием недостаточного обновления датасета.). При этом, половина фильмов в датасете сняты после 1997 года.

Изменение рейтинга фильмов в динамике по годам имеют обратнопропорциональную зависимость к количеству снимаемых фильмов. Так, прослеживается закономерность к снижению среднего рейтига с годами

In [11]:
count_year_df = df.groupby('year', as_index = False).filmtv_id.count()

trace = go.Bar(
    x = count_year_df.year,
    y = count_year_df.filmtv_id
)
layout = go.Layout(
    title='Кол-во фильмов по году выпуска ',
)

fig = go.Figure(data = [trace], layout = layout)
iplot(fig)

In [12]:
rating_year_df = df.groupby('year', as_index = False)[['avg_vote']].mean()

trace = go.Scatter(
    x = rating_year_df.year,
    y = rating_year_df.avg_vote,
    mode = 'lines',
    name = u'Средний рейтинг'
)

layout = go.Layout(
    title='Оценки фильмов по годам',
)   

fig = go.Figure(data = [trace], layout = layout)
iplot(fig)

Датасет представлен 27 жанрами, из них более половины имеет недостаточное количество записей, чтобы оценить статистическую закономерность рейтинга.

Из топ-10 жанров, наиболее высокий рейтинг показали документальные фильмы (6.8), анимационные (6.2) и драмы (6.2). Из не вошеших в топ жанров, наиболее высок средний рейтинг у жанров нуар (7.2), гангстерских (6.9), "мело" (6.9)

In [37]:
df.groupby('genre')[['filmtv_id']].count().sort_values('filmtv_id', ascending = False)

,filmtv_id
genre,
Drama,15833
Comedy,11569
Thriller,4851
Documentary,3138
Action,2857
Horror,2506
Adventure,2173
Fantasy,1603
Western,1342


In [39]:
def parse_list(lst_str):
    return filter(lambda y: y != '', 
                  map(lambda x: x.strip(), 
                      re.sub(r'[\[\]]', '', lst_str).split(',')))

df['genre'] = df['genre'].fillna('[]')
genre_data = []
for record in df.to_dict(orient = 'records'):
    genre_lst = parse_list(record['genre'])
    for genre in genre_lst:
        copy = record.copy()
        copy['genre'] = genre
        # copy['weight'] = 1./len(genre_lst)
        genre_data.append(copy)

genre_df = pd.DataFrame.from_dict(genre_data)

# сформируем список жанров
top_genres = genre_df.groupby('genre')[['filmtv_id']].count()\
    .sort_values('filmtv_id', ascending = False)\
    .head(10).index.values.tolist()

N = float(len(top_genres))

# cгенерируем цвета для визуализации
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]

data = [{
    'y': genre_df[genre_df.genre == top_genres[i]].avg_vote, 
    'type':'box',
    'marker':{'color': c[i]},
    'name': top_genres[i]
    } for i in range(len(top_genres))]

layout = go.Layout(
    title='Оценки фильмов по жанрам',
    yaxis = {'title': 'Средний рейтинг'}
)   

fig = go.Figure(data = data, layout = layout)
iplot(fig)

Оценивая фильмы по странам, наиболее высокие средние оценки получили картины из СССР (7.2), Польши (7.2), Бельгии (7.1),
далее следует Япония (7.0) и Южная Корея (7.0). 

Следует отметить, что российские фильмы на порядок уступают советскому кинематографу, и имеют среднюю оценку 6.7, что согласуется с общей тенденцией снижения рейтинга с годами.

In [45]:
country_count = df.groupby('country')[['filmtv_id']].count().sort_values('filmtv_id', ascending = False)
country_count.head(20)

,filmtv_id
country,
United States,21613
Italy,8907
France,2969
Great Britain,2476
Germany,1653
Japan,1098
Canada,1026
Spain,542
"Italy, France",396


In [47]:
def parse_list(lst_str):
    return filter(lambda y: y != '', 
                  map(lambda x: x.strip(), 
                      re.sub(r'[\[\]]', '', lst_str).split(',')))

df['country'] = df['country'].fillna('[]')
country_data = []
for record in df.to_dict(orient = 'records'):
    country_lst = parse_list(record['country'])
    for country in country_lst:
        copy = record.copy()
        copy['country'] = country
        # copy['weight'] = 1./len(genre_lst)
        country_data.append(copy)

country_df = pd.DataFrame.from_dict(country_data)

# сформируем список жанров
top_country = country_df.groupby('country')[['filmtv_id']].count()\
    .sort_values('filmtv_id', ascending = False)\
    .head(25).index.values.tolist()

N = float(len(top_country))

# cгенерируем цвета для визуализации
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]

data = [{
    'y': country_df[country_df.country == top_country[i]].avg_vote, 
    'type':'box',
    'marker':{'color': c[i]},
    'name': top_country[i]
    } for i in range(len(top_country))]

layout = go.Layout(
    title='Оценки фильмов по странам',
    yaxis = {'title': 'Средний рейтинг'}
)   

fig = go.Figure(data = data, layout = layout)
iplot(fig)

Оценивая диманику рейтинга каждого жанра в отдельности, можно отметить сохранение тенденции к снижению качества со временем.

Так, если вы будете смотреть фильм любого жанра, снятый до 1970 года, то высока вероятность, что он понравится больше аналогичной картины более позднего года.

In [23]:
genre_rating_year_df = genre_df.groupby(['year', 'genre'], as_index = False)[['avg_vote']].mean()

N = len(top_genres)
data = []
drop_menus = []

# конструируем все интересующие нас линии
for i in range(N):
    genre = top_genres[i]
    genre_df = genre_rating_year_df[genre_rating_year_df.genre == genre]

    trace = go.Scatter(
        x = genre_df.year,
        y = genre_df.avg_vote,
        mode = 'lines',
        name = genre,
        visible = (i == 0)
    )
    data.append(trace)

# создаем выпадающие меню
for i in range(N):
    drop_menus.append(
        dict(
            args=['visible', [False]*i + [True] + [False]*(N-1-i)],
            label= top_genres[i],
            method='restyle'
        )
    )

layout = go.Layout(
    title='Фильмы по жанрам',
    updatemenus=list([
        dict(
            x = -0.1,
            y = 1,
            yanchor = 'top',
            buttons = drop_menus
        )
    ]),
)

fig = go.Figure(data = data, layout = layout)
iplot(fig)

Оценивая качество кинематографа по странам, можно отметить, что страны с наиболее высоким средним рейтингом, отмеченные выше (СССР, Бельгия, Польша, Япония, Ю.Корея) продолжают получать высокие оценки зрителей на протяжении практически всего временного отрезка.

In [48]:
country_rating_year_df = country_df.groupby(['year', 'country'], as_index = False)[['avg_vote']].mean()

N = len(top_country)
data = []
drop_menus = []

# конструируем все интересующие нас линии
for i in range(N):
    country = top_country[i]
    country_df = country_rating_year_df[country_rating_year_df.country == country]

    trace = go.Scatter(
        x = country_df.year,
        y = country_df.avg_vote,
        mode = 'lines',
        name = country,
        visible = (i == 0)
    )
    data.append(trace)

# создаем выпадающие меню
for i in range(N):
    drop_menus.append(
        dict(
            args=['visible', [False]*i + [True] + [False]*(N-1-i)],
            label= top_country[i],
            method='restyle'
        )
    )

layout = go.Layout(
    title='Рейтинг фильмов по странам',
    updatemenus=list([
        dict(
            x = -0.1,
            y = 1,
            yanchor = 'top',
            buttons = drop_menus
        )
    ]),
)

fig = go.Figure(data = data, layout = layout)
iplot(fig)